In [1]:
import pandas as pd
df = pd.read_csv('Dataset/synthetic_logs.csv')
df.head(10)

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert


In [2]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [3]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [27]:
df

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


#### Finding Patterns for regex using clustering

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['log_message']) #list of embeddings


c:\Users\HARSH\OneDrive\Desktop\Projects\Log classifier\log_classification\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings #its an array we need to convert it to list
embeddings=embeddings.tolist()

In [6]:
embeddings[:1]

[[-0.10293962061405182,
  0.033545926213264465,
  -0.0220260601490736,
  0.0015510559314861894,
  -0.009869209490716457,
  -0.17895625531673431,
  -0.06344097852706909,
  -0.060176167637109756,
  0.028110884130001068,
  0.05996204540133476,
  -0.017261864617466927,
  0.0014336747117340565,
  -0.14956000447273254,
  0.003152857767418027,
  -0.05660310760140419,
  0.02716853842139244,
  -0.014989037998020649,
  -0.03540375456213951,
  -0.036293625831604004,
  -0.014541023410856724,
  -0.005614935420453548,
  0.08755391091108322,
  0.04551204293966293,
  0.025096368044614792,
  0.010018770582973957,
  0.012426701374351978,
  -0.13992361724376678,
  0.07686964422464371,
  0.03140952065587044,
  -0.0041524870321154594,
  0.0436902679502964,
  0.017125030979514122,
  -0.08009514957666397,
  0.05740059167146683,
  0.018909180536866188,
  0.0855262279510498,
  0.039639923721551895,
  -0.1343718320131302,
  -0.00144364929292351,
  0.003067078534513712,
  0.17685402929782867,
  0.004448867402970

In [7]:
#DBSCAN
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)


In [8]:
clusters

array([0, 1, 2, ..., 0, 1, 7], shape=(2410,))

In [9]:
df['clusters']=clusters

In [10]:
df.head()

,timestamp,source,log_message,target_label,complexity,clusters
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0


In [11]:
df[df.clusters==1]

,timestamp,source,log_message,target_label,complexity,clusters
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1
361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,bert,1
450,10/27/2025 5:59,ThirdPartyAPI,Email delivery system encountered an error,Error,bert,1
477,12/2/2025 10:30,AnalyticsEngine,Email transmission error caused service impact,Critical Error,bert,1
570,11/7/2025 18:08,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,bert,1
678,4/28/2025 15:13,AnalyticsEngine,Email delivery problem affected system,Critical Error,bert,1


In [12]:
#Sort clusters and only more important or populated once
cluster_cnt = df['clusters'].value_counts()
large_clusters = cluster_cnt[cluster_cnt>10].index

In [13]:
large_clusters

Index([ 0,  5, 11, 13,  7,  8, 21,  3,  4, 17,  6, 32, 16, 20,  9,  1, 10, 34,
       53, 14, 52, 18, 42, 25, 59, 26],
      dtype='int64', name='clusters')

In [14]:
for cluster in large_clusters:
    print(f"Cluster {cluster}:")
    print(df[df['clusters']==cluster]['log_message'].head(5).to_string(index=False))
    print()

Cluster 0:
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...

Cluster 5:
nova.compute.claims [req-a07ac654-8e81-416d-bfb...
nova.compute.claims [req-d6986b54-3735-4a42-907...
nova.compute.claims [req-72b4858f-049e-49e1-b31...
nova.compute.claims [req-5c8f52bd-8e3c-41f0-95a...
nova.compute.claims [req-d38f479d-9bb9-4276-968...

Cluster 11:
User User685 logged out.
 User User395 logged in.
 User User225 logged in.
User User494 logged out.
 User User900 logged in.

Cluster 13:
Backup started at 2025-05-14 07:06:55.
Backup started at 2025-02-15 20:00:19.
  Backup ended at 2025-08-08 13:06:23.
Backup started at 2025-11-14 08:27:43.
Backup started at 2025-12-09 10:19:11.

Cluster 7:
Multiple bad login attempts detected on user 85...
Multiple login failures occurred on user 9052 a...
  User 

In [15]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out)." : "User Action",
        r"Backup (started|ended) at .*." : "System Notification",
        r"Backup completed successfully." : "System Notification",
        r"System updated to version .*." : "System Notification",
        r"File .* uploaded successfully by user .*." : "System Notification",
        r"Disk cleanup completed successfully." : "System Notification",
        r"System reboot initiated by user .*." : "System Notification",
        r"Account with ID .* created by .*." : "User Action"
    }
    
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message):
            if re.search(pattern, log_message, re.IGNORECASE): # ignore small or capital differences
                return label
        return None

In [16]:
classify_with_regex("User User674 logged out.")

'User Action'

In [17]:
classify_with_regex("Use User674 logged out.")

In [18]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)

In [19]:
df.head(2)

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None


In [20]:
df[df.regex_label.notnull()] # 100 rows by regex only 

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
57,9/14/2025 3:03,AnalyticsEngine,User User395 logged in.,User Action,regex,11,User Action
85,3/13/2025 2:11,ModernHR,User User225 logged in.,User Action,regex,11,User Action
88,3/8/2025 19:04,AnalyticsEngine,User User494 logged out.,User Action,regex,11,User Action
126,11/22/2025 21:09,ThirdPartyAPI,User User900 logged in.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2207,10/4/2025 8:06,ModernCRM,User User495 logged in.,User Action,regex,11,User Action
2263,2/27/2025 14:40,AnalyticsEngine,User User429 logged out.,User Action,regex,11,User Action
2275,3/13/2025 17:17,AnalyticsEngine,User User755 logged out.,User Action,regex,11,User Action
2323,12/1/2025 18:17,ThirdPartyAPI,User User882 logged out.,User Action,regex,11,User Action


In [21]:
df_non_regex = df[df['regex_label'].isnull()].copy()

In [23]:
df_non_regex

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


Unknown pathway now:  
Enough training examples ===> BERT  
Not enough examples ===> LLM

In [25]:
df_non_regex['target_label'].value_counts()

target_label
HTTP Status            1017
Security Alert          371
System Notification     356
Error                   177
Resource Usage          177
Critical Error          161
User Action              44
Workflow Error            4
Deprecation Warning       3
Name: count, dtype: int64

In [24]:
print(df_non_regex['target_label'].value_counts()[df_non_regex['target_label'].value_counts()<=5].index.tolist())

['Workflow Error', 'Deprecation Warning']


In [ ]:
df_non_legacy = df_non_regex[df_non_regex.source != "LegacyCRM"]
df_non_legacy.source.unique() #source basedd filtering as source for above two less was same "LegacyCRM"


array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [33]:
df_non_legacy

,timestamp,source,log_message,target_label,complexity,clusters,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


BERT for sufficient example wale classifications

In [29]:
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())


In [32]:
filtered_embeddings[:2]

array([[-1.02939621e-01,  3.35459262e-02, -2.20260601e-02,
         1.55105593e-03, -9.86920949e-03, -1.78956255e-01,
        -6.34409785e-02, -6.01761676e-02,  2.81108841e-02,
         5.99620454e-02, -1.72618646e-02,  1.43367471e-03,
        -1.49560004e-01,  3.15285777e-03, -5.66031076e-02,
         2.71685384e-02, -1.49890380e-02, -3.54037546e-02,
        -3.62936258e-02, -1.45410234e-02, -5.61493542e-03,
         8.75539109e-02,  4.55120429e-02,  2.50963680e-02,
         1.00187706e-02,  1.24267014e-02, -1.39923617e-01,
         7.68696442e-02,  3.14095207e-02, -4.15248703e-03,
         4.36902680e-02,  1.71250310e-02, -8.00951496e-02,
         5.74005917e-02,  1.89091805e-02,  8.55262280e-02,
         3.96399237e-02, -1.34371832e-01, -1.44364929e-03,
         3.06707853e-03,  1.76854029e-01,  4.44886740e-03,
        -1.69275180e-02,  2.24266723e-02, -4.35049348e-02,
         6.09028572e-03, -9.98169184e-03, -6.23973049e-02,
         1.07372776e-02, -6.04895549e-03, -7.14660808e-0

Classify embeddings with logistic regression

In [35]:
X = filtered_embeddings
y = df_non_legacy['target_label']

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [39]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(x_test)
report = classification_report(y_test, y_pred)


In [40]:
print(report)

                     precision    recall  f1-score   support

     Critical Error       0.94      1.00      0.97        46
              Error       1.00      0.94      0.97        52
        HTTP Status       1.00      1.00      1.00       294
     Resource Usage       1.00      1.00      1.00        50
     Security Alert       1.00      1.00      1.00       120
System Notification       1.00      1.00      1.00       115
        User Action       1.00      1.00      1.00        14

           accuracy                           1.00       691
          macro avg       0.99      0.99      0.99       691
       weighted avg       1.00      1.00      1.00       691



In [46]:
import joblib
joblib.dump(clf, 'models/log_classifier.joblib')

['models/log_classifier.joblib']